In [ ]:
# C:\Users\<username>\.aws
# create file credentials без расширения
# вставьте туда то, что вы скопировали с AWS CLI:

In [1]:
import boto3
import json
from boto3.dynamodb.conditions import Key, Attr

In [70]:
# создаём подключение к клиенту базы данных
db_client = boto3.client('dynamodb')

###  Create table

In [6]:
# табличка
# при создании таблицы нужно указать только 2 ключа

table = db_client.create_table(
    TableName='Options',
    KeySchema=[
        {
            'AttributeName': 'user',
            'KeyType': 'HASH'  # Partition key основной ключ
        },
        {
            'AttributeName': 'date',
            'KeyType': 'RANGE'  # Sort key улюч сортировки
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'user',
            'AttributeType': 'S' # S значит стринг
        },
        {
            'AttributeName': 'date',
            'AttributeType': 'S'
        },
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 1,
        'WriteCapacityUnits': 1
    }
)

In [4]:
# вторая таблица
table = db_client.create_table(
    TableName='Choices',
    KeySchema=[
        {
            'AttributeName': 'user',
            'KeyType': 'HASH'  # Partition key
        },
        {
            'AttributeName': 'date',
            'KeyType': 'RANGE'  # Sort key
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'user',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'date',
            'AttributeType': 'S'
        },
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 1,
        'WriteCapacityUnits': 1
    }
)

In [72]:
# смотрим на все записи, которые есть в таблице
db_client.scan(TableName = "Choices")

{'Items': [{'user': {'S': 'vvseva'},
   'alternative': {'S': 'blue'},
   'date': {'S': '05/Dec/2021:12:54:32 +0000'},
   'selected': {'S': 'red'}},
  {'user': {'S': 'vvseva'},
   'alternative': {'S': 'yellow'},
   'date': {'S': '06/Dec/2021:12:28:44 +0000'},
   'selected': {'S': 'red'}},
  {'user': {'S': 'vvseva'},
   'alternative': {'S': 'yellow'},
   'date': {'S': '06/Dec/2021:12:29:13 +0000'},
   'selected': {'S': 'red'}},
  {'user': {'S': 'vvseva'},
   'alternative': {'S': 'blue'},
   'date': {'S': '06/Dec/2021:12:29:16 +0000'},
   'selected': {'S': 'green'}},
  {'user': {'S': 'vvseva'},
   'alternative': {'S': 'red'},
   'date': {'S': '06/Dec/2021:13:10:37 +0000'},
   'selected': {'S': 'yellow'}},
  {'user': {'S': 'vvseva'},
   'alternative': {'S': 'yellow'},
   'date': {'S': '06/Dec/2021:13:10:39 +0000'},
   'selected': {'S': 'red'}},
  {'user': {'S': 'vvseva'},
   'alternative': {'S': 'red'},
   'date': {'S': '06/Dec/2021:13:10:55 +0000'},
   'selected': {'S': 'yellow'}},
  {'us

In [17]:
# по другому подключаемся к 1 таблице, чтобы делать запросы к ней
dynamodb = boto3.resource('dynamodb')
table_Choices = dynamodb.Table('Choices')
table_Choices

dynamodb.Table(name='Choices')

### Read table

In [58]:
# поиск по полю user чтобы оно было равно нашему значению

user = 'vvseva'
response = table_Choices.query(
    KeyConditionExpression=Key('user').eq(user),
    ScanIndexForward = False,
    Limit = 1
)

In [64]:
# парсим джос
response['Items'][0]

'vvseva'

In [65]:
table_Choices = dynamodb.Table('Options')
user = 'vvseva'
response = table_Choices.query(
    KeyConditionExpression=Key('user').eq(user),
    ScanIndexForward = False,
    Limit = 1
)

In [73]:
response['Items'][0]

{'user': 'vvseva',
 'date': '06/Dec/2021:13:00:38 +0000',
 'option1': 'yellow',
 'option0': 'red'}

## INVOKE labmda
используем лямбду без апи
такой же алгоритм если хотите использовать лямбду внутри другой лямбды

In [73]:
import typing, json
# подключаетесь к клиету лямбд
lambda_client = boto3.client('lambda')

In [74]:

def invokeLambdaFunction(*, functionName:str=None, payload:typing.Mapping[str, str]=None):
    '''

    :param functionName: имя функции
    :param payload: словарь (джсон) с аргументами
    :return: результат из лямбды. скорее всего джсон
    '''
    if  functionName == None:
        raise Exception('ERROR: functionName parameter cannot be NULL')
    payloadStr = json.dumps(payload)
    payloadBytesArr = bytes(payloadStr, encoding='utf8')
    client = boto3.client('lambda')
    response = client.invoke(
        FunctionName=functionName,
        InvocationType="RequestResponse",
        Payload=payloadBytesArr
    )
    return response

In [ ]:
# функция cph из примера в конфле

In [84]:
payloadObj = {"text" : "how are you doing LOVE YOU"}
response = invokeLambdaFunction(functionName='cph',  payload=payloadObj)

In [85]:
response

{'ResponseMetadata': {'RequestId': 'ec05bb66-ed79-440c-9f5a-a12d7f79872b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 06 Dec 2021 17:58:42 GMT',
   'content-type': 'application/json',
   'content-length': '510',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ec05bb66-ed79-440c-9f5a-a12d7f79872b',
   'x-amzn-remapped-content-length': '0',
   'x-amz-executed-version': '$LATEST',
   'x-amzn-trace-id': 'root=1-61ae4f52-22f7b0081d014f88296be37f;sampled=0'},
  'RetryAttempts': 0},
 'StatusCode': 200,
 'ExecutedVersion': '$LATEST',
 'Payload': <botocore.response.StreamingBody at 0x21e6b335af0>}

In [86]:
# парсим ответ
response_json = json.loads(response["Payload"].read().decode("utf-8"))
response_json

{'statusCode': 200,
 'body': {'Sentiment': 'POSITIVE',
  'SentimentScore': {'Positive': 0.9199503660202026,
   'Negative': 0.004127929452806711,
   'Neutral': 0.07217232882976532,
   'Mixed': 0.003749364987015724},
  'ResponseMetadata': {'RequestId': 'c414e54c-bae9-488f-8455-c9234587f4ae',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amzn-requestid': 'c414e54c-bae9-488f-8455-c9234587f4ae',
    'content-type': 'application/x-amz-json-1.1',
    'content-length': '164',
    'date': 'Mon, 06 Dec 2021 17:58:42 GMT'},
   'RetryAttempts': 0}}}

In [82]:
response_json['body']['Sentiment']

'NEGATIVE'

In [83]:
response_json['body']['SentimentScore']

{'Positive': 0.0008534153457731009,
 'Negative': 0.9878454804420471,
 'Neutral': 0.010970450937747955,
 'Mixed': 0.00033070490462705493}

## S3

In [2]:
resource = boto3.resource('s3')

In [ ]:
# https://mybacketaws.s3.amazonaws.com/chicago_parks_reviews.csv

In [3]:
my_bucket = resource.Bucket('mybacketaws')
key = 'chicago_parks_reviews.csv'
local_filename = 'chicago_parks_reviews.csv'

In [4]:
my_bucket.download_file(key, local_filename)

link be like  s3://mybucket/foo/bar/file.txt

Bucket   --> mybucket
Key      --> foo/bar/file.txt
Filename --> /local/path/file.txt

In [ ]:
Bucket = 'mybacketaws'

my_bucket = resource.Bucket(Bucket)
key = 'chicago_parks_reviews.csv'
local_filename = 'test_data_m3.csv'

In [ ]:
my_bucket.download_file(key, local_filename)